# Introduction to Python HT20


##### Import Libraries and Load Files

In [5]:
import os
import gzip
import itertools
import time

# Path to notebook
notebook_path = os.path.abspath("mainAssignment.ipynb")
print(notebook_path)

# Human chromosome 7 reference sequence
chr7_ref = "data/Homo_sapiens.GRCh38.dna_sm.chromosome.7.fa.gz"
# Human reference genome annotation
annotation= "data/Homo_sapiens.GRCh38.93.gtf"

/Users/kbenevides/python_workshop_course/mainAssignment.ipynb


### WARM UP 

Two short excercises to get started! 

##### 1. What is the length of chromosome 7 on the reference sequence?

In [57]:

def count_chr_1(file):
    start_time=time.time()
    
    lines = gzip.open(file).read().splitlines()[1:]
    num = sum([len(i) for i in lines])
    
    end_time=time.time()
    run= end_time - start_time
    print("Method 1 %s seconds" % round(run,3))
    return num

# method 2
def count_chr_2(file):
    start_time = time.time()
    
    num = 0
    with gzip.open(file) as fasta:
        for line in itertools.islice(fasta,1,None):
            num += len(line.strip())
    
    end_time = time.time()
    run= end_time - start_time
    print("Method 2 %s seconds" % round(run,3))
    return num

#method 3
def count_chr_3(file):
    start_time = time.time()
    
    f = gzip.open(file, 'r')
    length = 0
    for line in f:
        if not line.startswith(b">"):
            length += len(line.strip())  # add the length of the line, without newline character
    f.close()
    
    end_time = time.time()
    run= end_time - start_time
    print("Method 3 %s seconds" % round(run,3))
    return length


print("The length of the human chromosome 7 is: %d nt" % count_chr_1(chr7_ref))
print("The length of the human chromosome 7 is: %d nt" % count_chr_2(chr7_ref))
print("The length of the human chromosome 7 is: %d nt" % count_chr_3(chr7_ref))

Method 1 1.53 seconds
The length of the human chromosome 7 is: 159345973 nt
Method 2 3.386 seconds
The length of the human chromosome 7 is: 159345973 nt
Method 3 3.676 seconds
The length of the human chromosome 7 is: 159345973 nt


##### 2. How many genes are annotated in the GTF file? 

In [58]:
def count_genes(file):
    genes = 0
    f = open(file)
    for line in f:
        if not line.startswith('#'):
            feature = line.split('\t')[2]
            if feature == "gene":
                genes += 1
    return genes

print("Number of genes in the GTF file: %d" % count_genes(annotation))

Number of genes in the GTF file: 58395


### ASSIGNMENT 

The main assignment consists of 6 subtasks to architect a method for finding the patients at risk. 

#### 1. How many transcripts can the CTFR gene generate?

In [59]:
def find_transcripts(file, gene):
    transcripts = 0
    f = open(file)
    for line in f:
        if not line.startswith('#'):
            gene_id = line.split('\t')[8].split(";")[0].split(" ")[1]
            feature = line.split('\t')[2]
            if gene in gene_id:
                if feature == "transcript":
                    transcripts += 1
    return transcripts

print("Number of transcripts for the CTFR gene: %d" % find_transcripts(annotation, "ENSG00000001626"))           
            
    

Number of transcripts for the CTFR gene: 11


#### 2. Which of these transcripts is the longest transcript in nucleotides?

In [3]:
def transcript_len(file, gene): 
    transcripts = []
    lengths = []
    f = open(file)
    for line in f:
        if not line.startswith('#'):
            feature = line.split('\t')[2]
            gene_id = line.split('\t')[8].split(";")[0].split(" ")[1]
            trans_id = line.split('\t')[8].split(";")[2].split(" ")[2]
            if gene in gene_id:
                if feature == "transcript":
                    lengths.append(int(line.split('\t')[4]) - int(line.split('\t')[3]))
                    transcripts.append(trans_id)
    ind = lengths.index(max(lengths))
    return transcripts[ind], lengths[ind]

print("The longest transcript(s) for the CTFR gene is: %s which is %d nucelotides long" % (transcript_len(annotation, "ENSG00000001626")[0], int(transcript_len(annotation, "ENSG00000001626")[1])))           


The longest transcript(s) for the CTFR gene is: "ENST00000003084" which is 188702 nucelotides long


#### 3. Fetch the DNA sequence for that transcript

In [6]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from utils import check_answers

def extract_seq(annotation, fasta, transcript, outfile): 
    """
    Given a reference annotation file and a transcript ID, this function retrieves 
    the DNA sequence for that transcript. 
    
    Required parameters: 
    
    [1] Annotation file 
    [2] Reference fasta file
    [3] Transcript ID
    [4] Directory to output file
    """
    with open(annotation, "rt") as gtf: 
        for line in gtf:
            if not line.startswith('#'):
                feature = line.split('\t')[2]
                trans_id = line.split('\t')[8].split(";")[2].split(" ")[2]
                if transcript in trans_id:
                    if feature == "transcript": 
                        (start, stop) = (int(line.split('\t')[3]), int(line.split('\t')[4]))
    
    with gzip.open(fasta, "rt") as f:
        for rec in SeqIO.parse(f, "fasta"):
            trans_seq_record = SeqRecord(Seq(str(rec.seq))[start:stop], id=transcript, description='')    
        SeqIO.write(trans_seq_record, outfile, "fasta")

# Arguments 
transcript = "ENST00000003084"
transcript_outfile = "data/transcript.fasta"

extract_seq(annotation, chr7_ref, transcript, transcript_outfile)

# Check answer
check_answers.ex3(transcript_outfile)

The sequences are of different length, try again


#### 4. Fetch the DNA sequences of all exons for that transcript, spliced together to one sequence.

In [6]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import defaultdict
from utils import check_answers

def extract_exon(annotation, fasta, transcript, outfile): 
    """
    Given a reference annotation file and a transcript ID, this function retrieves 
    the DNA sequence for that transcript. 
    
    Required parameters: 
    
    [1] Annotation file 
    [2] Reference fasta file
    [3] Transcript ID
    [4] Directory to output file
    """
    with open(annotation, "rt") as gtf:
        exon_stop_start = defaultdict(list)
        for line in gtf:
            if not line.startswith('#'):
                feature = line.strip().split('\t')[2]
                trans_id = line.strip().split('\t')[8].split(";")[2].split(" ")[2] 
                if transcript in trans_id:
                    if feature == "exon":
                        exon_id =line.strip().split('\t')[8].split(";")[13].split(" ")[2]
                        start = int(line.strip().split('\t')[3])
                        stop = int(line.strip().split('\t')[4])
                        exon_stop_start[exon_id].append([start,stop])
                                    
    with gzip.open(fasta, "rt") as f:
        seqs = ""
        for rec in SeqIO.parse(f, "fasta"):
            for keys, values in exon_stop_start.items():
                seqs += Seq(str(rec.seq))[values[0][0]-1:values[0][1]]
            exon_record = SeqRecord(seqs, id=transcript + "_exons", description='')

            
        SeqIO.write(exon_record, outfile, "fasta")

# Arguments 
transcript = "ENST00000003084"
exon_outfile = "data/exons.fasta"

extract_exon(annotation, chr7_ref, transcript, exon_outfile)

# Check answer
#check_answers.ex4(exon_outfile)

#### 5. What are the start positions and sequences of the start_codon and stop_codon for that transcript?

In [31]:

def extract_stop_start(annotation, fasta, transcript):
    """
    Given a reference annotation file and a transcript ID, this function retrieves 
    the DNA sequence for that transcript. 
    
    Required parameters: 
    
    [1] Annotation file 
    [2] Reference fasta file
    [3] Transcript ID
    """
    with open(annotation, "rt") as gtf:
        exon_stop_start = defaultdict(list)
        for line in gtf:
            if not line.startswith('#'):
                feature = line.strip().split('\t')[2]
                trans_id = line.strip().split('\t')[8].split(";")[2].split(" ")[2] 
                if transcript in trans_id:
                    if feature == "start_codon":
                        start_codon = int(line.strip().split('\t')[3])
                    if feature == "stop_codon": 
                        stop_codon = int(line.strip().split('\t')[3])
    
    with gzip.open(fasta, "rt") as f:
        seqList = [] # store the DNA sequence of chromosome 7 in a list
        for line in f:
            if not line.startswith(">"):
                seqList.append(line.strip())
        chr7 = "".join(seqList) # join all DNA fragments from the seqList to one string
        

    if chr7[start_codon-1:start_codon+2] == "ATG":
        print("The start codon is", chr7[start_codon-1:start_codon+2], "and starts at position", start_codon)
    else:
        print("The start codon is not ATG")

    if chr7[stop_codon-1:stop_codon+2] == "TAG" or chr7[stop_codon-1:stop_codon+2] == "TAA" or chr7[stop_codon-1:stop_codon+2] == "TGA":
        print("The stop codon is", chr7[stop_codon-1:stop_codon+2], "and starts at position", stop_codon)
    else:
        print("The stop codon does not correspond to a proper stop codon")

# Arguments 
transcript = "ENST00000003084"
exon_infile = "data/exons.fasta"

extract_stop_start(annotation, chr7_ref, transcript)


The start codon is ATG and starts at position 117480095
The stop codon is TAG and starts at position 117667106


6. Translate the above sequence of all exons into amino acids, using an implementation of the translation table from the utils.rna package 

In [47]:
from utils import rna 

# Get the longest transcript and its exons, as well as the start codon
g = open(annotation, 'r')
gene_id = "ENSG00000001626"
transcript = '"ENST00000003084"'

exons = []

for line in g:
    if not line.startswith("#"):
        entries = line.strip().split("\t")
        if gene_id in line:
            if entries[2] == "exon":
                attribute = entries[8].split(";")
                transcriptID = attribute[2].strip().split(' ')[1]
                if transcript == transcriptID:
                    start = int(entries[3])
                    end = int(entries[4])
                    exons.append((start, end))
            elif entries[2] == 'start_codon':
                attribute = entries[8].split("; ")
                transcriptID = attribute[2].strip().split(' ')[1]
                if transcript == transcriptID:
                    start_codon = int(entries[3])

print("The first exon has the positions", exons[0])
print("The start codon has the position", start_codon)
g.close()

# Extract all exons of the longest transcript sequence from the genome fasta file

with gzip.open(chr7_ref, "rt") as f:
    seqList = [] # store the DNA sequence of chromosome 7 in a list
    for line in f:
        if not line.startswith(">"):
            seqList.append(line.strip())
    chr7 = "".join(seqList) # join all DNA fragments from the seqList to one string
    f.close()

mRNA = "" # add exon sequences to string
for exon in exons:
    mRNA += chr7[exon[0]-1:exon[1]]

# Translate to aminoacids, from the correct start position on
if chr7[start_codon-1:start_codon+2] != "ATG":
    print("The start codon is not ATG")
else:
    start = start_codon - exons[0][0] # get position of start codon in mRNA by subtracting the start codon position and the start position of the first exon

from utils.rna import translate_dna
aminoacids = translate_dna(mRNA[start:]) # translate the mRNA sequence from the start codon on
print(aminoacids[:10], "...")

a = open("data/CFTR_longest_transcript_aminoacids.txt", 'w')
a.write(aminoacids)
a.close()

The first exon has the positions (117479963, 117480147)
The start codon has the position 117480095
MQRSPLEKAS ...


### Finding the patients at risk

A mutation in the transcript ENST00000003084 causes a premature stop codon to be introduced into the amino acid sequence. This creates a truncated protein, causing cystic fibrosis. Use the python code you have written to solve the tasks above and extend it to compare the reference genome sequence of chromosome 7 to the sequences of the following 5 patients in fasta format (Patient1.fa.gz, Patient2.fa.gz, Patient3.fa.gz, Patient4.fa.gz, and Patient5.fa.gz) to determine which of them are carrying a mutation in the CFTR gene that causes a truncated protein.

In [54]:
from utils import rna 

# Get the longest transcript and its exons, as well as the start codon
g = open(annotation, 'r')
gene_id = "ENSG00000001626"
transcript = '"ENST00000003084"'

exons = []

for line in g:
    if not line.startswith("#"):
        entries = line.strip().split("\t")
        if gene_id in line:
            if entries[2] == "exon":
                attribute = entries[8].split(";")
                transcriptID = attribute[2].strip().split(' ')[1]
                if transcript == transcriptID:
                    start = int(entries[3])
                    end = int(entries[4])
                    exons.append((start, end))
            elif entries[2] == 'start_codon':
                attribute = entries[8].split("; ")
                transcriptID = attribute[2].strip().split(' ')[1]
                if transcript == transcriptID:
                    start_codon = int(entries[3])
g.close()

# Extract all exons of the longest transcript sequence from the genome fasta file

with gzip.open(chr7_ref, "rt") as f:
    seqList = [] # store the DNA sequence of chromosome 7 in a list
    for line in f:
        if not line.startswith(">"):
            seqList.append(line.strip())
    chr7 = "".join(seqList) # join all DNA fragments from the seqList to one string
    f.close()

mRNA = "" # add exon sequences to string
for exon in exons:
    mRNA += chr7[exon[0]-1:exon[1]]

# Translate to aminoacids, from the correct start position on
if chr7[start_codon-1:start_codon+2] != "ATG":
    print("The start codon is not ATG")
else:
    start = start_codon - exons[0][0] # get position of start codon in mRNA by subtracting the start codon position and the start position of the first exon

from utils.rna import translate_dna
aminoacids = translate_dna(mRNA[start:]) # translate the mRNA sequence from the start codon on

i = 1 # index for loop through patient fasta files
while i < 6:
    p = gzip.open("data/Patient" + str(i) + ".fa.gz", 'rt')
    patientSeqList = [] # store the DNA sequence of the patient in a list
    for line in p:
        if not line.startswith(">"):
            patientSeqList.append(line.strip())
    patientSeq = "".join(patientSeqList) # construct one sequence for the currently analyzed patient

    patientmRNA = "" # append exon sequences to string. Works only for small strings.
    for exon in exons:
        patientmRNA += patientSeq[exon[0]-1:exon[1]]

    # Translate to aminoacids
    patient_aminoacids = translate_dna(patientmRNA[start:]) # translate the mRNA sequence from the start codon on
    if not len(patient_aminoacids) == len(aminoacids):
        print("The sequence of patient ", str(i), " has a different length than the reference genome sequence.")

    p.close()
    i += 1

The sequence of patient  3  has a different length than the reference genome sequence.
